In [12]:
import pandas as pd
import numpy as np
from datetime import datetime

from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, HoverTool, Legend, LegendItem, Title, CategoricalTicker
from bokeh.transform import factor_cmap
from bokeh.io import output_notebook, export_png, export_svg

import assets

output_notebook()

Loading BokehJS ...

## WOW 2018

### Week 01: Looks vs. Personality

In [3]:
df08w01 = pd.read_csv(assets.DATA_18W01).query('Measure=="Ranked personality higher than looks"')
df08w01.head()

,Nationality,Gender,Value,Measure
2,Danish,Men,77,Ranked personality higher than looks
6,Danish,Women,93,Ranked personality higher than looks
10,Finnish,Men,77,Ranked personality higher than looks
14,Finnish,Women,93,Ranked personality higher than looks
18,British,Men,80,Ranked personality higher than looks


In [22]:
df08w01_filtered = df08w01.iloc[:, [0,1,2]]
SIZE = 18
CIRCLE_COLOR = '#449EA0'

source = ColumnDataSource(df08w01_filtered)

filter_cond = df08w01_filtered['Nationality'].isin(['Egyptian', 'Saudi Arabian', 'Vietnamese'])
line_source = ColumnDataSource(df08w01_filtered.loc[filter_cond].pivot(
    index='Nationality', columns='Gender', values='Value'
).reset_index())

y_range = df08w01_filtered.query('Gender=="Women"').sort_values(by="Value")['Nationality']

p = figure(
    y_range=y_range,
    x_range=[0, 100],
    height=700,
)

p.segment(
    x0='Men', x1='Women', 
    y0='Nationality', y1='Nationality', 
    line_width=2, color='black',
    source=line_source
)

circle = p.scatter(
    x='Value', y='Nationality',
    size=SIZE, #legend_field='Gender',
    line_color=CIRCLE_COLOR, line_width=3,
    fill_color=factor_cmap('Gender', palette=[CIRCLE_COLOR, 'white'], factors=['Men', 'Women']),
    source=source
)

hover = HoverTool(tooltips="<b>@Nationality</b><br>@Gender: <b>@Value</b>", renderers=[circle])
p.add_tools(hover)

legend = Legend(
    items=[
        LegendItem(label="Men", renderers=[circle], index=0),
        LegendItem(label="Women", renderers=[circle], index=1)
        ], 
    orientation="horizontal", 
    background_fill_alpha=0,
    border_line_alpha=0,
    spacing=40,
    location="top_center",
)

title = Title(
    text="Across the world women are more likely than men to value\npersonality over looks",
    text_font_size="20px",
    offset=-55,
    standoff=10,
)
subtitle = Title(
    text=("% of people who ranked a romantic partner having a personality "
          "they liked as more important\nthan them being good looking"),
    offset=-55,
    standoff=-5,
    text_font_style="normal"
)

p.add_layout(legend, 'above')
p.add_layout(subtitle, 'above')
p.add_layout(title, 'above')


show(p)

In [ ]:
CategoricalTicker()